In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!sudo apt-get install espeak-ng
!pip install TTS
!pip install pydub
!pip install trainer==0.0.20

In [ ]:
# import needed libraries
import os
import torch
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.bin.compute_embeddings import compute_embeddings
from TTS.tts.utils.speakers import SpeakerManager, get_speaker_balancer_weights, get_speaker_manager

In [ ]:
%%capture
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav
model_file="/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth"

In [ ]:
output_path="/content/drive/MyDrive/vits/traineroutput/"

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="vctk", meta_file_train="", phonemizer="espeak", dataset_name="vctk-vits", language="en-us", path="path/to/data"
)

In [ ]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

vitsArgs = VitsArgs(
    use_speaker_embedding=True,
)

config = VitsConfig(
    model_args=vitsArgs,
    audio=audio_config,
    run_name="tts_multispeaker",
    batch_size=16,
    eval_batch_size=16,
    eval_split_max_size=256,
    eval_split_size=0.01,
    batch_group_size=5,
    num_loader_workers=2,
    num_eval_loader_workers=2,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    max_text_len=500,  # change this if you have a larger VRAM than 16GB
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    save_best_after=1000,
    save_all_best=True,
    save_checkpoints=True,
    save_n_checkpoints=50,
    save_step=1000,
    max_audio_len=22050 * 15
)

In [ ]:
%%capture
ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

In [ ]:
speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.model_args.num_speakers = speaker_manager.num_speakers
config.num_speakers=speaker_manager.num_speakers

In [ ]:
model = Vits(config, ap, tokenizer, speaker_manager)

In [ ]:
RESTORE_PATH="/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth"

In [ ]:
trainer = Trainer(
    TrainerArgs(restore_path=RESTORE_PATH, skip_train_epoch=False),
    config,
    output_path=output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

In [ ]:

%load_ext tensorboard
%tensorboard --logdir "/path/to/log/dir"

In [ ]:
trainer.fit()